In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import requests
import pandas as pd

# Set the parameters for the API request
api_key = 'd7fb0a055cfe4ca0a152'  # Your API key
service_id = 'COOKRCP01'
data_type = 'json'
start_idx = '1'
end_idx = '1000'

# Construct the API request URL
url = f"http://openapi.foodsafetykorea.go.kr/api/{api_key}/{service_id}/{data_type}/{start_idx}/{end_idx}"

# Make the request
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON data
    data = response.json()
    recipes_data = data['COOKRCP01']['row']

    # Define the columns you want to extract
    columns = ['RCP_SEQ', 'RCP_NM', 'RCP_WAY2', 'RCP_PAT2', 'INFO_WGT', 'INFO_ENG', 'INFO_CAR', 'INFO_PRO', 'INFO_FAT', 'INFO_NA', 'HASH_TAG', 'ATT_FILE_NO_MAIN', 'ATT_FILE_NO_MK', 'RCP_PARTS_DTLS']
    manual_steps = [f'MANUAL{i:02}' for i in range(1, 21)]
    manual_img_steps = [f'MANUAL_IMG{i:02}' for i in range(1, 21)]
    columns += manual_steps + manual_img_steps

    # Create a DataFrame
    df = pd.DataFrame(recipes_data, columns=columns)

    # Select a specific column from the DataFrame
    specific_column = df.iloc[:, 15]  # For example, selecting the 16th column

    # Define new column names
    new_column_names = ['일련번호', '메뉴명', '조리방법', '요리종류', '중량(1인분)', '열량', '탄수화물', '단백질', '지방', '나트륨', '해쉬태그', '이미지경로(소)', '이미지경로(대)', '재료정보'] + [f'만드는법_{i:02}' for i in range(1, 21)] + [f'만드는법_이미지_{i:02}' for i in range(1, 21)]

    # Ensure the length of 'new_column_names' matches the number of columns in your DataFrame
    assert len(new_column_names) == len(df.columns)

    # Assign the new column names to the DataFrame
    df.columns = new_column_names
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")


In [ ]:
#df.iloc[:,16]

0      3. 시금치를 잘게 다져 혼합물 그릇(2)에 뿌리고 찜기에 넣고 중간 불에서 10분...
1      3. 저염간장에 다진 대파, 다진 마늘, 고춧가루, 요리당 , 참기름, 참깨를 섞어...
2                    3. 칼집을 낸 방울토마토에 양념속을 사이사이에 넣어 버무린다.
3      3. 썰어 놓은 오이에 순두부사과 소스를 넣고 버무린 후 다진 땅콩을 뿌려 마무리 한다.
4      3. 찬물에 북어채, 새우, 표고버섯, 양파를 넣고 20분 정도 끓인 후 사과를 넣...
                             ...                        
995    3. 달걀흰자는 머랭을 만들어\n2번과 섞은 뒤 크랜베리, 잘라둔\n고구마와 살살 ...
996    3. 체에 내린 가루에 삶은 고구마,\n감말랭이, 냉동딸기, 흑임자를\n넣고 믹서에...
997    3. 타르트팬에 버터를 바른 뒤\n반죽을 얇게 깔고 그 위에\n바나나를 담고 소금을...
998    3. 냄비에 포도즙, 꿀, 물, 소금을\n넣고 끓으면 불을 끄고 식용유를\n넣고 섞는다.
999      3. 휴지시킨 반죽을 밀대로 밀어\n커터로 모양을 찍은 뒤 180℃\n오븐에 굽는다.
Name: 만드는법_03, Length: 1000, dtype: object

In [ ]:
#recipe_step = df.loc[df['메뉴명'] == '사과 새우 북엇국', '만드는법_02']
#print(recipe_step)


4    2. 북어채를 잘게 손으로 찢어 찬물에 헹구어 준비한 후 양파, 표고버섯, 사과는 ...
Name: 만드는법_02, dtype: object


In [ ]:
# '만드는법_01'부터 '만드는법_20'까지를 합쳐 '레시피' 열 생성
df['레시피'] = df.apply(lambda row: ' '.join([str(row[f'만드는법_{i:02}']) for i in range(1, 21) if pd.notnull(row[f'만드는법_{i:02}'])]), axis=1)

# '만드는법_01'부터 '만드는법_20'까지의 열 삭제
for i in range(1, 21):
    column_name = f'만드는법_{i:02}'
    if column_name in df.columns:
        df.drop(column_name, axis=1, inplace=True)


In [ ]:
for i in range(1, 21):
    column_name = f'만드는법_이미지_{i:02}'
    if column_name in df.columns:
        df.drop(column_name, axis=1, inplace=True)


In [ ]:
df.loc[df['메뉴명'] == '사과 새우 북엇국']

,일련번호,메뉴명,조리방법,요리종류,중량(1인분),열량,탄수화물,단백질,지방,나트륨,해쉬태그,이미지경로(소),이미지경로(대),재료정보,레시피
4,33,사과 새우 북엇국,끓이기,국&찌개,,65,2,12,1,78,,http://www.foodsafetykorea.go.kr/uploadimg/coo...,http://www.foodsafetykorea.go.kr/uploadimg/coo...,"북엇국\n북어채 25g(15개), 새우 10g(3마리), 사과 30g(1/5개), ...","2. 북어채를 잘게 손으로 찢어 찬물에 헹구어 준비한 후 양파, 표고버섯, 사과는..."


In [ ]:
# Save the DataFrame as a CSV file to Google Drive
file_path = '/content/drive/MyDrive/GDSC/Solution Challenge/food recipe.csv'  # Specify your desired file name
df.to_csv(file_path, index=False)  # 'index=False' to exclude row indices from the CSV file